# Simulating a virtual Soft X-ray Tomography system

In [ ]:
# imports
from utils import *
from scipy.stats import lognorm
from scipy.signal import convolve2d

In [2]:
# parameters
KEEP_ONLY_TO_KEEP = True # [bool] keep only the rays kept by Luca O. in the real dataset
# dataset parameters
POLAR = False # use polar coordinates

# constants for random gaussians
MAX_MIX = 1 # max number of gaussians
KR = (0.1, 0.8) if POLAR else (0.0, 0.9)  # [m] max radius mean multiplier
KXY = (1/10, 1/4) # std deviation range multiplier

# N = 500_000 # train
# N = 100_000 # train
# N = 50_000 # train
# N = 10_000 # train
N = 1000

In [3]:
# convert from polar to cartesian coordinates and vice versa
def polar2cart(v): 
    vshape = v.shape
    v = v.reshape(-1, 2)
    r, θ = v[:,0], v[:,1]
    x, y = RM+r*cos(θ), ZM+r*sin(θ)
    xy = np.stack((x, y), axis=-1)
    return xy.reshape(vshape)

def cart2polar(v): 
    vshape = v.shape
    v = v.reshape(-1, 2)
    x, y = v[:,0], v[:,1]
    r, θ = hypot(x-RM, y-ZM), arctan2(y-ZM, x-RM)
    rθ = np.stack((r, θ), axis=-1)
    return rθ.reshape(vshape)

In [ ]:
# test polar2cart and cart2polar
RZ_POL = cart2polar(RZ)
assert RZ.shape == RZ_POL.shape, f'Expected {RZ.shape} but got {RZ_POL.shape}'
assert np.allclose(RZ_POL, cart2polar(polar2cart(RZ_POL))), 'cart2polar(polar2cart(x)) != x'
assert np.allclose(RZ, polar2cart(cart2polar(RZ))), 'polar2cart(cart2polar(x)) != x'
# plot RZ_POL
plt.figure(figsize=(6,6))
plt.scatter(*RZ_POL.T, s=5, c='r')
plt.title('RZ_POL')
plt.xlabel('R')
plt.ylabel('Z')
plt.show()

In [ ]:
# test vertical line
c, θ = (2, 0), π/2
# c, θ = (2, 0), π/6

l1 = create_line(c, θ)
print(l1.shape)

# lmask = line_mask(c, 0, θ)
mask, midxs = line_mask(c, 0, θ)
# mask, midxs = lmask
print(np.sum(mask))

square_mask = np.zeros((GSIZE*GSIZE))
square_mask[midxs] = mask

# plot
plt.figure()
# plt.scatter(l1[:,0], l1[:,1], c='r', s=1)
plt.scatter(RR, ZZ, c=square_mask.reshape((GSIZE,GSIZE)), s=10, cmap='gray')
plt.plot(l1[:,0], l1[:,1], ':r')
plt.plot(FW[:,0], FW[:,1], 'w')
plt.xlim(R0-GSPAC/2, R1+GSPAC/2)
plt.ylim(Z0-GSPAC/2, Z1+GSPAC/2)
plt.gca().set_aspect('equal', adjustable='box')
plt.grid(False)
plt.colorbar()
plt.show()

In [ ]:
# test line
c, θ = (2, 0), π/2
c, θ = (2, 0), uniform(0, π)
# c, θ = (2, 0), π/6

l1 = create_line(c, θ)
print(l1.shape)

# lmask = line_mask(c, 0, θ)
mask, midxs = line_mask(c, 0, θ)
# mask, midxs = lmask
print(np.sum(mask))

square_mask = np.zeros((GSIZE*GSIZE))
square_mask[midxs] = mask

# plot
plt.figure()
# plt.scatter(l1[:,0], l1[:,1], c='r', s=1)
plt.scatter(RR, ZZ, c=square_mask.reshape((GSIZE,GSIZE)), s=10, cmap='gray')
plt.plot(l1[:,0], l1[:,1], ':r')
plt.plot(FW[:,0], FW[:,1], 'w')
plt.xlim(R0-GSPAC/2, R1+GSPAC/2)
plt.ylim(Z0-GSPAC/2, Z1+GSPAC/2)
plt.gca().set_aspect('equal', adjustable='box')
plt.grid(False)
plt.colorbar()
plt.show()

In [ ]:
# test polar gaussian and rays
# #standard gaussian
# μ = np.array([R0+2*L/3, Z0+3*L/5])
# Σ = np.array([[L/40, 0], [0, L/16]])
# gauss = gaussian(RZ, μ, Σ) 

# polar gaussian
# μ, Σ = np.array([0.25, π/3]), np.array([[1/100, 0.0], [0.0, π/3]])
# μ, Σ = np.array([0.0, π/3]), np.array([[1/500, 0.0], [0.0, 100*π]])
μ, Σ = np.array([0.2, π/3]), np.array([[1/500, 0.0], [0.0, π/5]])
gauss_pol = gaussian(RZ_POL, μ, Σ, polar=True)
gauss = gauss_pol
# c, θ = (2, 0), π/6
# c, θ = (2, 0), uniform(0, π)
c, θ = (uniform(1.5,2.5), uniform(-.5, .5)), uniform(0, π)
mask1, midxs1 = line_mask(c, 0, θ)
square_mask1 = np.zeros((GSIZE*GSIZE))
square_mask1[midxs1] = mask1

combined = gauss.reshape(-1)[midxs1] * mask1 * GSPAC

combined_square = np.zeros((GSIZE*GSIZE))
combined_square[midxs1] = combined

# sxr integration values
sxr = np.sum(combined)
print(f'SXR: {sxr:.4f}')

# plot the 4 maps
#plot gasussian polar
plt.figure(figsize=(20,5))
plt.subplot(141)
plt.scatter(*RZ_POL.T, s=5, c=gauss_pol)
plt.title('Gaussian Polar')
plt.xlabel('r'), plt.ylabel('θ')
plt.colorbar()
plt.subplot(142)
plt.scatter(RR, ZZ, c=gauss, s=20)
plt.clim(0, 1)
plt.title('Gaussian')
plt.axis('equal')
plt.colorbar()
plt.grid(False)
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.subplot(143)
plt.scatter(RR, ZZ, c=square_mask1, s=10)
plt.clim(0, 1)
plt.title('Line')
plt.axis('equal')
plt.colorbar()
plt.grid(False)
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.subplot(144)
plt.scatter(RR, ZZ, c=combined_square/GSPAC, s=10)
plt.clim(0, 1)
plt.title('Combined')
plt.axis('equal')
plt.colorbar()
plt.grid(False)
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.show()

In [ ]:
# create RFX fans of rays 
μ = np.array([R0+2*L/3, Z0+3*L/5])
Σ = np.array([[L/16, 0], [0, L/40]])
gauss = gaussian(RZ, μ, Σ) 
# nrays = [VDI_NRAYS, VDC_NRAYS, VDE_NRAYS, HOR_NRAYS]
# αs_start = [VDI_START_ANGLE, VDC_START_ANGLE, VDE_START_ANGLE, HOR_START_ANGLE]
# αs_span = [VDI_SPAN_ANGLE, VDC_SPAN_ANGLE, VDE_SPAN_ANGLE, HOR_SPAN_ANGLE]
# pinholes = [VDI_PINHOLE_POS, VDC_PINHOLE_POS, VDE_PINHOLE_POS, HOR_PINHOLE_POS]
# to_keeps = [VDI_TO_KEEP, VDC_TO_KEEP, VDE_TO_KEEP, HOR1_TO_KEEP]
sxrs, rayss, fans, inc_αs = [], [], [], [] # sxr, rays, fans, incidence angles
for name, n, α_start, α_span, p, c, keep in zip(RFX_SXR_NAMES, RFX_SXR_NRAYS, RFX_SXR_STARTS, RFX_SXR_SPANS, RFX_SXR_PINHOLES, RFX_SXR_COLORS, RFX_SXR_TO_KEEP):
    start_time = time()
    rays, fan, αs_incidence = create_rfx_fan(n, α_start, α_span, p, keep, ret_all=True) # 0.1 s at gs=55
    combined_square = np.zeros((len(fan), GSIZE*GSIZE))
    for i, (mask, midxs) in enumerate(fan):
        combined_square[i,midxs] = gauss.reshape(-1)[midxs] * mask * GSPAC
    sxr = np.sum(combined_square, axis=-1)
    inc_αs.append(αs_incidence)
    rayss.append(rays)
    fans.append(fan)
    sxrs.append(sxr)
    plt.figure(figsize=(15,3))
    plt.subplot(131)
    plt.scatter(RR, ZZ, c=gauss, s=900/GSIZE, marker='s')
    plt.clim(0, 1)
    for r in rays: plt.plot(r[:,0], r[:,1], f'{c}')
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.title(f'{name} Rays')
    plt.grid(False)
    plt.subplot(132)
    plt.scatter(RR, ZZ, c=combined_square.sum(axis=0)/GSPAC, s=6*150/GSIZE, marker='s')
    plt.axis('equal')
    # plt.clim(0, 1)
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.grid(False)
    plt.colorbar()
    plt.title(f'{name} Rays integration')
    plt.subplot(133)
    plt.plot(αs_incidence, sxr/GSPAC, f'{c}s-')
    plt.xticks([-π/4, -π/8, 0, π/8, π/4], ['-π/4', '-π/8', '0', 'π/8', 'π/4'])
    plt.grid(True)
    plt.ylim(0, 1)
    plt.title(f'{name} SXR')
    plt.show()

# plot everything together
plt.figure(figsize=(12,4))
plt.subplot(121)
plt.scatter(RR, ZZ, c=gauss, s=6000/GSIZE, marker='s')
plt.clim(0, 1)
for rs, c in zip(rayss, RFX_SXR_COLORS):
    for r in rs: plt.plot(r[:,0], r[:,1], f'{c}')
plt.axis('equal')
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.colorbar()
plt.title('Rays')
plt.grid(False)
plt.subplot(122)
for θs, c, sxr, s, α, in zip(inc_αs, RFX_SXR_COLORS, sxrs, RFX_SXR_STARTS, RFX_SXR_SPANS):
    plt.plot(θs, sxr/GSPAC, f'{c}s-')
plt.xticks([-π/4, -π/8, 0, π/8, π/4], ['-π/4', '-π/8', '0', 'π/8', 'π/4'])
plt.grid(True)
plt.ylim(0, 1)
plt.title('SXR')
plt.show()

# Create a Dataset

In [9]:
# helper functions
def create_random_means_stds(max_mix=MAX_MIX, kr=KR, kxy=KXY, polar=POLAR):
    n = np.random.randint(1, max_mix+1) if max_mix > 1 else 1 # number of gaussians
    # mix = uniform(kmix[0], kmix[1], n) # mixing coefficients
    mix = lognorm.rvs(*MAX_EMISS_LOGNORM_PARAMS, size=n) # mixing coefficients
    μr = uniform(L*kr[0]/2, L*kr[1]/2, n) # [m] random radius
    μθ = uniform(0, 2*π, n) # [rad] random angle
    μ = np.stack((μr, μθ), axis=-1) # [m, rad] random mean
    Σ = np.zeros((n, 2, 2)) # covariance matrix
    if polar:
        # kθ = (μr*kxy[0], μr*kxy[1]) # [rad] random angle std deviation
        # Σ[:,1,1] = uniform(kθ[0], kθ[1], n)**2
        Σ[:,0,0] = uniform(L*kxy[0], L*kxy[1], n)**2
        Σ[:,1,1] = uniform(L*2*π*kxy[0], L*2*π*kxy[1], n)**2
    else:
        μ = polar2cart(μ) #convert to cartesian 
        Σ[:,0,0] = uniform(L*kxy[0], L*kxy[1], n)**2
        Σ[:,1,1] = uniform(L*kxy[0], L*kxy[1], n)**2
    return mix, μ, Σ

def eval_gaussians(ps, mix, μ, Σ, polar=False):
    assert len(mix) == len(μ) == len(Σ), 'mix, μ, Σ must be lists of the same length'
    d = np.zeros(ps.shape[:-1]) # initialize the distribution
    if polar: ps = cart2polar(ps) # convert to polar
    for i in range(len(μ)): d += mix[i]*gaussian(ps, μ[i], Σ[i], polar) # add the gaussians
    return d

def create_random_gaussian_mix():
    mix, μ, Σ = create_random_means_stds()
    g = eval_gaussians(RZ, mix, μ, Σ, POLAR)
    return g

def create_polygon_distribution(n=3, levels=10):
    # rc, θc, θr = uniform(0, L/3), uniform(0, 2*π), uniform(0, 2*π) # center of the polygon
    rc, θc, θr = uniform(0, L/3), uniform(0, 2*π), 0 # center of the polygon
    cx, cz = RM+rc*cos(θc), ZM+rc*sin(θc) # center of the polygon
    rp = uniform(L/3, L/2-rc) # radius of the polygon
    θs = np.linspace(0, 2*π, n, endpoint=False) + θr # angles of the vertices
    rs = np.linspace(rp/levels, rp, levels, endpoint=False) # radii of the vertices
    polys = [np.array([np.array([cx+r*cos(θ), cz+r*sin(θ)]) for θ in θs]) for r in rs] # create the polygon
    
    def are_inside(ps, poly):
        n = len(poly)
        inside = np.zeros(len(ps), dtype=bool)
        x1s, y1s = poly[:,0], poly[:,1]
        x2s, y2s = np.roll(x1s, -1), np.roll(y1s, -1)
        xps, yps = ps[:,0], ps[:,1]
        for i in range(n):
            x1, y1 = x1s[i], y1s[i]
            x2, y2 = x2s[i], y2s[i]
            if y1 > y2: x1, y1, x2, y2 = x2, y2, x1, y1
            idxs = (y1 <= yps) & (yps < y2) & ((yps-y1)*(x2-x1) >= (xps-x1)*(y2-y1))
            inside[idxs] = ~inside[idxs]            
        return inside

    d = np.zeros(GSIZE*GSIZE)
    for poly in reversed(polys):
        d += are_inside(RZ.reshape(-1, 2), poly)/levels
    d = d.reshape(GSIZE, GSIZE)

    # apply a smoothing filter
    d = convolve2d(d, np.ones((3,3))/9, mode='same', boundary='wrap').reshape(-1)


    # # plot the polygon
    # plt.figure(figsize=(6,6))
    # plt.scatter(*RZ.T, s=15, c=d)
    # plt.title('Polygon Distribution')
    # plt.xlabel('R')
    # plt.ylabel('Z')
    # plt.show()


    return d.reshape(GSIZE,GSIZE)

In [ ]:
# test
mix, μ, Σ = create_random_means_stds(MAX_MIX, KR, KXY, POLAR) # create random means and stds
g = eval_gaussians(RZ, mix, μ, Σ, POLAR) # evaluate the gaussians
rfx_sxr = RFX_SXR(GSIZE) # load the rfx fans
sxrs = [rfx_sxr.eval_on_fan(g, f) for f in rfx_sxr.fan_names] # evaluate the SXR on the fans

# plot g and sxr
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.scatter(RR, ZZ, c=g, s=1500/GSIZE, marker='s')
# plot rays
for rs, c in zip(rayss, RFX_SXR_COLORS):
    for r in rs: plt.plot(r[:,0], r[:,1], f'{c}')
plt.axis('equal')
plt.xlim(R0, R1), plt.ylim(Z0, Z1)
plt.colorbar()
plt.title('Distribution HR')
plt.grid(False)
plt.subplot(122)
for c, sxr, a in zip(RFX_SXR_COLORS, sxrs, inc_αs):
    plt.plot(a, sxr, f'{c}s-')
plt.grid(True)
plt.legend()
plt.title('SXR')
plt.show()

In [ ]:
# create dataset (N train samples, N//10 test/validation samples)
def create_dataset(n):
    emiss = np.zeros((n, GSIZE, GSIZE), dtype=np.float32) # emissivity
    vdi = np.zeros((n, len(fans[0]),), dtype=np.float32) 
    vdc = np.zeros((n, len(fans[1]),), dtype=np.float32) 
    vde = np.zeros((n, len(fans[2]),), dtype=np.float32)
    hor = np.zeros((n, len(fans[3]),), dtype=np.float32)
    rfx_sxr = RFX_SXR(GSIZE) # load the rfx fans
    for i in tqdm(range(n), desc=f'Creating {n} samples'):
        # mix, μ, Σ = create_random_means_stds(MAX_MIX, KR, KXY, POLAR)
        # emiss[i] = eval_gaussians(RZ, mix, μ, Σ, POLAR)
        emiss[i] = create_polygon_distribution(6, 15)
        vdi[i] = rfx_sxr.eval_on_fan(emiss[i], 'vdi')
        vdc[i] = rfx_sxr.eval_on_fan(emiss[i], 'vdc')
        vde[i] = rfx_sxr.eval_on_fan(emiss[i], 'vde')
        hor[i] = rfx_sxr.eval_on_fan(emiss[i], 'hor')
    # save the dataset
    np.savez(f'data/sxr_sim_ds_gs{GSIZE}_n{n}.npz', emiss=emiss, vdi=vdi, vdc=vdc, vde=vde, hor=hor, RR=RR, ZZ=ZZ)

create_dataset(N)
create_dataset(N//10)
# create_dataset(N//100)

In [ ]:
# load the dataset
data = np.load(f'data/sxr_sim_ds_gs{GSIZE}_n{N//10}.npz')
print(data.files)
# extract the data
emiss, vdi, vdc, vde, hor = data['emiss'], data['vdi'], data['vdc'], data['vde'], data['hor']
print(emiss.shape, vdi.shape, vdc.shape, vde.shape, hor.shape)

In [ ]:
# plot the dataset
N_PLOTS = 10
idxs = np.random.randint(0, N//10, N_PLOTS)
for i in idxs:
    plt.figure(figsize=(10,3))
    mind, maxd = np.min(emiss[i]), np.max(emiss[i])
    # distribution
    plt.subplot(121)
    plt.scatter(RR, ZZ, c=emiss[i], s=6*150/GSIZE, marker='s')
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.clim(mind, maxd)
    plt.title(f'Distribution {i}')
    plt.grid(False)
    # sxr
    plt.subplot(122)
    plt.plot(inc_αs[0], vdi[i], f'{RFX_SXR_COLORS[0]}s:', label='VDI')
    plt.plot(inc_αs[1], vdc[i], f'{RFX_SXR_COLORS[1]}s:', label='VDC')
    plt.plot(inc_αs[2], vde[i], f'{RFX_SXR_COLORS[2]}s:', label='VDE')
    plt.plot(inc_αs[3], hor[i], f'{RFX_SXR_COLORS[3]}s:', label='HOR')
    plt.xticks([-π/4, -π/8, 0, π/8, π/4], ['-π/4', '-π/8', '0', 'π/8', 'π/4'])
    plt.grid(True)
    plt.legend()
    plt.title(f'SXR {i}')
    plt.show()
    plt.close()

## Test the simulated tomography system on real distributions


In [ ]:
# test of simulation on real distributions
data = np.load(f'data/sxr_real_ds_gs{GSIZE}_n{1000}.npz')
print(data.files)
# extract the data
emiss, vdi, vdc, vde, hor = data['emiss'], data['vdi'], data['vdc'], data['vde'], data['hor']
print(emiss.shape, vdi.shape, vdc.shape, vde.shape, hor.shape)
n = emiss.shape[0]

# load the rfx fans
rfx_sxr = RFX_SXR(GSIZE) # load the rfx fans

# plot the dataset
N_PLOTS = 20
idxs = np.random.randint(0, n, N_PLOTS)
for i in idxs:
    # now lets calculate the SXR from the real distributions
    sim_vdi = rfx_sxr.eval_on_fan(emiss[i], 'vdi')
    sim_vdc = rfx_sxr.eval_on_fan(emiss[i], 'vdc')
    sim_vde = rfx_sxr.eval_on_fan(emiss[i], 'vde')
    sim_hor = rfx_sxr.eval_on_fan(emiss[i], 'hor')
    assert len(sim_vdi) == len(vdi[i]), 'VDI length mismatch'
    assert len(sim_vdc) == len(vdc[i]), 'VDC length mismatch'
    assert len(sim_vde) == len(vde[i]), 'VDE length mismatch'
    assert len(sim_hor) == len(hor[i]), 'HOR length mismatch'

    plt.figure(figsize=(15,6))
    mind, maxd = np.min(emiss[i]), np.max(emiss[i])
    # distribution
    plt.subplot(231)
    plt.scatter(RR, ZZ, c=emiss[i], s=6*150/GSIZE, marker='s')
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.clim(mind, maxd)
    plt.title(f'Distribution {i}')
    plt.grid(False)
    # contour
    plt.subplot(232)
    plt.contour(RR, ZZ, emiss[i], 20)
    plt.axis('equal')
    plt.xlim(R0, R1), plt.ylim(Z0, Z1)
    plt.colorbar()
    plt.clim(mind, maxd)
    plt.grid(False)
    plt.title(f'Distribution {i}')
    # plot the SXR real vs simulated
    plt.subplot(233) # vdi
    plt.plot(inc_αs[0], vdi[i], 'rs:', label='VDI Real')
    plt.plot(inc_αs[0], sim_vdi, 'r-', label='VDI Sim')
    plt.xticks([-π/16, -π/32, 0, π/32, π/16], ['-π/16', '-π/32', '0', 'π/32', 'π/16'])
    plt.grid(True)
    plt.legend()
    plt.title(f'VDI {i}')
    plt.subplot(234) # vdc
    plt.plot(inc_αs[1], vdc[i], 'gs:', label='VDC Real')
    plt.plot(inc_αs[1], sim_vdc, 'g-', label='VDC Sim')
    plt.xticks([-π/8, -π/16, 0, π/16, π/8], ['-π/8', '-π/16', '0', 'π/16', 'π/8'])
    plt.grid(True)
    plt.legend()
    plt.title(f'VDC {i}')
    plt.subplot(235) # vde
    plt.plot(inc_αs[2], vde[i], 'bs:', label='VDE Real')
    plt.plot(inc_αs[2], sim_vde, 'b-', label='VDE Sim')
    plt.xticks([-π/8, -π/16, 0, π/16, π/8], ['-π/8', '-π/16', '0', 'π/16', 'π/8'])
    plt.grid(True)
    plt.legend()
    plt.title(f'VDE  {i}')
    plt.subplot(236) # hor
    plt.plot(inc_αs[3], hor[i], 'ys:', label='HOR Real')
    plt.plot(inc_αs[3], sim_hor, 'y-', label='HOR Sim')
    plt.xticks([-π/4, -π/8, 0, π/8, π/4], ['-π/4', '-π/8', '0', 'π/8', 'π/4'])
    plt.grid(True)
    plt.legend()
    plt.title(f'HOR {i}')

    plt.show()
    plt.close()